# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission, if necessary. Sections that begin with **'Implementation'** in the header indicate where you should begin your implementation for your project. Note that some sections of implementation are optional, and will be marked with **'Optional'** in the header.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

Let's import the libraries we'll need.

In [ ]:
import tensorflow as tf
import pickle
import numpy as np
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline
import matplotlib.image as mpimg
from itertools import groupby
import csv
from sklearn.model_selection import train_test_split

---
## Step 0: Load The Data

In [ ]:
# Load pickled data
training_file = "../traffic-signs-data/train.p"
testing_file = "../traffic-signs-data/test.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 2D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below.

In [ ]:
# Number of training examples
n_train = X_train.shape[0]
# Number of testing examples.
n_test = X_test.shape[0]

# What's the shape of an traffic sign image?
image_shape = X_train[0].shape

# How many unique classes/labels there are in the dataset.
n_classes = len(set(y_train))

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc.

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections.

Let's print the name of each image class. This will help us identify how many images we have per sign type.

In [ ]:
def load_labels(csv_name):
    d = {}
    with open(csv_name) as csvfile:
        reader = csv.DictReader(csvfile)
        d = {int(row['ClassId']): row['SignName'] for row in reader}
    return d

def print_legend(csv_name):
    with open(csv_name) as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            print(row['ClassId'], ':', row['SignName'])
            
print_legend('signnames.csv')

In [ ]:
# These are the helper functions used for visualization and data exploration            
def plot_bar_chart(dataset, title):
    # Group the dataset.
    labels = load_labels('signnames.csv')
    data = {(k, labels[k]): len(list(g)) for k, g in groupby(sorted(dataset[:]))}
    
    # Make plot and put the proper labels.
    plt.figure(figsize=(6,16))
    plt.barh(range(len(data)), data.values(), color='cyan')
    
    plt.ylabel("Image class")
    plt.xlabel("Number of images")
    
    plt.title(title)
    plt.yticks(range(len(data)), sorted(data, key=lambda) 
    # Show plot
    plt.show()
    
def show_sample_images(X, y, img_class, num_samples=5):
    assert(0 <= img_class <= 42)
    imgs_of_class = X[y == img_class]
    
    sample = np.random.choice(len(imgs_of_class), num_samples)
    for img in imgs_of_class[sample]:
        plt.subplot(1, 2, 1)
        plt.imshow(img)
        plt.show()

**Training** set exploration:

In [ ]:
plot_bar_chart(y_train, "Train labels")
show_sample_images(X_train, y_train, img_class=0, num_samples=2)

**Test** set exploration:

In [ ]:
plot_bar_chart(y_test, "Test labels")
show_sample_images(X_test, y_test, img_class=0, num_samples=1)

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

There are various aspects to consider when thinking about this problem:

- Neural network architecture
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

**NOTE:** The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play!

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:
### Preprocess the data here.
### Feel free to use as many code cells as needed.
X_train = X_train.astype(float)
X_train -= np.mean(X_train, axis = 0) # zero-center
X_train /= np.std(X_train, axis = 0) # normalize
X_test = X_test.astype(float)
X_test -= np.mean(X_test, axis = 0) # zero-center
X_test /= np.std(X_test, axis = 0) # normalize

### Question 1 

_Describe how you preprocessed the data. Why did you choose that technique?_

**Answer:**

In [ ]:
### Generate data additional data (OPTIONAL!)
### and split the data into training/validation/testing sets here.
### Feel free to use as many code cells as needed.

def randomize(dataset, labels):
    permutation = np.random.permutation(labels.shape[0])
    shuffled_dataset = dataset[permutation, :, :]
    shuffled_labels = labels[permutation]
    return shuffled_dataset, shuffled_labels

X_train, y_train = randomize(X_train, y_train)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.15, random_state=42)

# Let's print some statistics again:
print("TRAINING examples: ", X_train.shape[0])
print("VALIDATION examples: ", X_valid.shape[0])
print("TEST examples: ", X_test.shape[0])
print("EXAMPLES shape: ", X_train.shape)
print("LABELS shape: ", y_train.shape)

show_sample_images(X_train, y_train, img_class=1, num_samples=1)

### Question 2

_Describe how you set up the training, validation and testing data for your model. **Optional**: If you generated additional data, how did you generate the data? Why did you generate the data? What are the differences in the new dataset (with generated data) from the original dataset?_

**Answer:**

Let's use a LeNet. Here's the function that creates the layers:

In [ ]:
from tensorflow.contrib.layers import flatten

# Hyperparameters
mu = 0
sigma = 0.1

def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape, mean=mu, stddev=sigma))

def bias_variable(shape):
    return tf.Variable(tf.zeros(shape))

# LeNet architecture:
# INPUT -> CONV -> ACT -> POOL -> CONV -> ACT -> POOL -> FLATTEN -> FC -> ACT -> FC
def LeNet(x):
    # Reshape from 2D to 4D. This prepares the data for
    # convolutional and pooling layers.
    x = tf.reshape(x, (-1, 32, 32, 3))
    # Pad 0s to 32x32. Centers the digit further.
    # Add 2 rows/columns on each side for height and width dimensions.
    #x = tf.pad(x, [[0, 0], [2, 2], [2, 2], [0, 0]], mode="CONSTANT")
    
    # TODO: Change to 32x32x12
    # Convolution layer 1. The output shape should be 28x28x6
    cv1_filter_width = 5
    cv1_filter_height = 5
    cv1_output_depth = 6
    cv1_weight = weight_variable([cv1_filter_width, cv1_filter_height, 3, cv1_output_depth])
    cv1_bias = bias_variable(cv1_output_depth)
    cv1_layer = tf.nn.conv2d(x, cv1_weight, strides=[1,1,1,1], padding='VALID')
    cv1_layer = tf.nn.bias_add(cv1_layer, cv1_bias)
    cv1_layer = tf.nn.relu(cv1_layer)

    # TODO: Change to 16x16x12
    # Pooling layer 1. The output shape should be 14x14x6.
    p1_filter_width = 2
    p1_filter_height = 2
    p1_output_depth = 6
    p1_layer = tf.nn.max_pool(cv1_layer, ksize=[1, p1_filter_width, p1_filter_height, 1], strides=[1,1,1,1], padding='VALID')

    # TODO: Change to 10x10x24
    # Convolution layer 2. The output shape should be 10x10x16
    cv2_filter_width = 5
    cv2_filter_height = 5
    cv2_output_depth = 16
    cv2_weight = weight_variable([cv2_filter_width, cv2_filter_height, p1_output_depth, cv2_output_depth])
    cv2_bias = bias_variable(cv2_output_depth)
    cv2_layer = tf.nn.conv2d(p1_layer, cv2_weight, strides=[1,1,1,1], padding='VALID')
    cv2_layer = tf.nn.bias_add(cv2_layer, cv2_bias)
    cv2_layer = tf.nn.relu(cv2_layer)

    # TODO: Change to 5x5x24
    # Pooling layer 2. The output shape should be 5x5x16.
    p2_filter_width = 2
    p2_filter_height = 2
    p2_layer = tf.nn.max_pool(cv2_layer, ksize=[1, p2_filter_width, p2_filter_height, 1], strides=[1,1,1,1], padding='VALID')

    # Flatten layer
    flatten_layer = flatten(p2_layer)

    # TODO: Change to 240
    fc1_outputs = 120
    fc1_weigth = weight_variable([flatten_layer.get_shape().as_list()[-1], fc1_outputs])
    fc1_biases = bias_variable([fc1_outputs])
    fc1_layer = tf.add(tf.matmul(flatten_layer, fc1_weigth), fc1_biases)
    fc1_layer = tf.nn.relu(fc1_layer)

    fc2_outputs = n_classes
    fc2_weight = weight_variable([fc1_outputs, fc2_outputs])
    fc2_biases = bias_variable([fc2_outputs])
    fc2_layer = tf.add(tf.matmul(fc1_layer, fc2_weight), fc2_biases)

    # Return the result of the last fully connected layer.
    return fc2_layer

Our hyperparams will be:

In [ ]:
BETA = 1e-3
BATCH_SIZE = 50
EPOCHS = 10
n_input = 32 * 32 * 3 

Our model:

In [ ]:
X = tf.placeholder(tf.float32, shape=[None, n_input])
y = tf.placeholder(tf.float32, shape=[None, n_classes])
logits = LeNet(X)

Optimizer:

In [ ]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, y))
optimizer = tf.train.AdamOptimizer().minimize(loss)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

### Question 3

_What does your final architecture look like? (Type of model, layers, sizes, connectivity, etc.)  For reference on how to build a deep neural network using TensorFlow, see [Deep Neural Network in TensorFlow
](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/b516a270-8600-4f93-a0a3-20dfeabe5da6/concepts/83a3a2a2-a9bd-4b7b-95b0-eb924ab14432) from the classroom._


**Answer:**

First, let's onehot encode our labels:

In [ ]:
def reshape_input(data):
    return data.reshape((-1, n_input))

def onehot_encode(data):
    return tf.Session().run(tf.one_hot(data, n_classes))
X_train_flat = reshape_input(X_train)
X_valid_flat = reshape_input(X_valid)
X_test_flat = reshape_input(X_test)
y_train_onehot = onehot_encode(y_train)
y_valid_onehot = onehot_encode(y_valid)
y_test_onehot = onehot_encode(y_test)

In [ ]:
### Train your model here.
### Feel free to use as many code cells as needed.

def eval_dataset(data, labels):
    n_examples = data.shape[0]
    total_batches = n_examples // BATCH_SIZE
    
    total_accuracy = 0 
    total_loss = 0
    
    for i in range(total_batches):
        offset = (i * BATCH_SIZE)
        batch_x = data[offset:offset + BATCH_SIZE, :]
        batch_y = labels[offset:offset + BATCH_SIZE, :]
        l, acc = session.run([loss, accuracy], feed_dict={X: batch_x, y: batch_y})
        
        total_accuracy += (acc * batch_x.shape[0])
        total_loss += (l * batch_x.shape[0])
    
    return total_loss / n_examples, total_accuracy / n_examples
    
init = tf.global_variables_initializer()
loss_acc = []
train_acc = []
valid_acc = []
with tf.Session() as session:
    session.run(init)
    
    for epoch in range(EPOCHS):
        total_batches = X_train.shape[0] // BATCH_SIZE
        
        for i in range(total_batches):
            offset = (i * BATCH_SIZE)
            batch_x = X_train_flat[offset:offset + BATCH_SIZE, :]
            batch_y = y_train_onehot[offset:offset + BATCH_SIZE, :]
            session.run(optimizer, feed_dict={X: batch_x, y: batch_y})
        
        
        training_loss, training_accuracy = eval_dataset(X_train_flat, y_train_onehot)
        loss_acc.append(training_loss)
        train_acc.append(training_accuracy)
        
        validation_loss, validation_accuracy = eval_dataset(X_valid_flat, y_valid_onehot)
        valid_acc.append(validation_accuracy)
        
        
        if epoch % 1 == 0:
            print("EPOCH:", epoch)
            print("Loss:", training_loss)
            print("Training accuracy:", training_accuracy * 100)
            print("Validation accuracy:", validation_accuracy * 100)
            print("------------------------------------------\n")
            
    
    test_loss, test_accuracy = eval_dataset(X_test_flat, y_test_onehot) 
    print("Test accuracy:", test_accuracy * 100)

epochs_range = range(EPOCHS)
plt.plot(epochs_range, loss_acc)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

plt.plot(epochs_range, train_acc)
plt.plot(epochs_range, valid_acc)
plt.xlabel("Epochs")
plt.ylabel("Training accuracy - Validation accuracy")
plt.show()

### Question 4

_How did you train your model? (Type of optimizer, batch size, epochs, hyperparameters, etc.)_


**Answer:**

### Question 5


_What approach did you take in coming up with a solution to this problem? It may have been a process of trial and error, in which case, outline the steps you took to get to the final solution and why you chose those steps. Perhaps your solution involved an already well known implementation or architecture. In this case, discuss why you think this is suitable for the current problem._

**Answer:**

---

## Step 3: Test a Model on New Images

Take several pictures of traffic signs that you find on the web or around you (at least five), and run them through your classifier on your computer to produce example results. The classifier might not recognize some local signs but it could prove interesting nonetheless.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Question 6

_Choose five candidate images of traffic signs and provide them in the report. Are there any particular qualities of the image(s) that might make classification difficult? It could be helpful to plot the images in the notebook._



**Answer:**

In [ ]:
### Run the predictions here.
### Feel free to use as many code cells as needed.

### Question 7

_Is your model able to perform equally well on captured pictures when compared to testing on the dataset? The simplest way to do this check the accuracy of the predictions. For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate._

_**NOTE:** You could check the accuracy manually by using `signnames.csv` (same directory). This file has a mapping from the class id (0-42) to the corresponding sign name. So, you could take the class id the model outputs, lookup the name in `signnames.csv` and see if it matches the sign from the image._


**Answer:**

In [ ]:
### Visualize the softmax probabilities here.
### Feel free to use as many code cells as needed.

### Question 8

*Use the model's softmax probabilities to visualize the **certainty** of its predictions, [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. Which predictions is the model certain of? Uncertain? If the model was incorrect in its initial prediction, does the correct prediction appear in the top k? (k should be 5 at most)*

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

**Answer:**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.